In [21]:
import os
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz
import plotly.io as pio

In [22]:
# Reading the dataset
app_data = pd.read_csv("C:/Users/Ayush Singh/NullClass/google_play_store_data/googleplaystore.csv")
print("App Data Columns:", app_data.columns)

App Data Columns: Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')


In [23]:
# Clean 'Size' column, replace 'Varies with device' with None, and converting to numeric
app_data['Size'] = app_data['Size'].replace('Varies with device', None)
app_data['Size'] = app_data['Size'].str.replace('M', '').str.replace('k', '')
app_data['Size'] = pd.to_numeric(app_data['Size'], errors='coerce')

In [24]:
# Converting 'Last Updated' to datetime
app_data['Last Updated'] = pd.to_datetime(app_data['Last Updated'], errors='coerce')

In [25]:
# Converting 'Reviews' column to numeric
app_data['Reviews'] = pd.to_numeric(app_data['Reviews'], errors='coerce')

In [26]:
# Filtering the data
df_filtered = app_data[ 
    ~(app_data['Rating'] < 4.0) & 
    ~(app_data['Reviews'] < 10) & 
    (app_data['App'].str.contains('C', case=False, na=False))
]

In [27]:
# Filter by category with more than 50 apps
category_counts = df_filtered['Category'].value_counts()
valid_categories = category_counts[category_counts > 50].index
df_filtered = df_filtered[df_filtered['Category'].isin(valid_categories)]

In [28]:
# Getting the remaining categories after filtering
remaining_categories = df_filtered['Category'].unique()
print("Remaining Categories:", remaining_categories)

Remaining Categories: ['BOOKS_AND_REFERENCE' 'BUSINESS' 'COMMUNICATION' 'DATING' 'EDUCATION'
 'FINANCE' 'FOOD_AND_DRINK' 'HEALTH_AND_FITNESS' 'LIFESTYLE' 'GAME'
 'FAMILY' 'MEDICAL' 'SOCIAL' 'SHOPPING' 'PHOTOGRAPHY' 'SPORTS'
 'TRAVEL_AND_LOCAL' 'TOOLS' 'PERSONALIZATION' 'PRODUCTIVITY'
 'NEWS_AND_MAGAZINES']


In [29]:
# Counting the number of apps per category in the filtered dataset
category_counts_remaining = df_filtered['Category'].value_counts()
print("Remaining Categories and their app counts:\n", category_counts_remaining)

Remaining Categories and their app counts:
 Category
FAMILY                 679
GAME                   431
TOOLS                  291
PHOTOGRAPHY            175
PERSONALIZATION        164
COMMUNICATION          162
PRODUCTIVITY           162
SPORTS                 156
FINANCE                147
HEALTH_AND_FITNESS     145
MEDICAL                135
BUSINESS               131
SOCIAL                 127
SHOPPING               113
LIFESTYLE              102
DATING                  90
TRAVEL_AND_LOCAL        87
EDUCATION               83
NEWS_AND_MAGAZINES      71
BOOKS_AND_REFERENCE     61
FOOD_AND_DRINK          51
Name: count, dtype: int64


In [30]:
# all categories are included, even those with no data after filtering
all_categories = app_data['Category'].unique()

In [31]:
# Current time
now_ist = datetime.now(pytz.timezone('Asia/Kolkata'))
current_hour = now_ist.hour

In [32]:
# Checking current time is between 4 PM and 6 PM
if 16 <= current_hour < 18:
    # Creating violin plot 
    fig_violin = px.violin(
        df_filtered,
        x='Category',
        y='Rating',
        color='Category',
        box=True, 
        points='all', 
        title='Distribution of Ratings for App Categories (Filtered)',
        labels={"Category": "App Category", "Rating": "App Rating"},
        category_orders={"Category": sorted(remaining_categories)}, 
        template='plotly'
    )

    # Layout for the violin plot
    fig_violin.update_layout(
        title_font_size=16,
        xaxis_title_font_size=12,
        yaxis_title_font_size=12,
        xaxis_tickangle=45,
        xaxis_tickfont=dict(size=10),
        yaxis_tickfont=dict(size=10),
        showlegend=False
    )

    # Path to save HTML and PNG files
    html_files_path = "./"  
    if not os.path.exists(html_files_path):
        os.makedirs(html_files_path)
    
    # Save as HTML
    file_path_html = os.path.join(html_files_path, "Violin_Plot.html")
    fig_violin.write_html(file_path_html)

    # Save as PNG
    file_path_png = os.path.join(html_files_path, "Violin_Plot.png")
    pio.write_image(fig_violin, file_path_png)

    # Show
    fig_violin.show()

else:
    print("The graph is only available between 4 PM and 6 PM IST.")

The graph is only available between 4 PM and 6 PM IST.
